Here is your **fully rewritten, professional, clean, and integrated Project Description** — now perfectly aligned with your dataset, ACS tables, and the four-category + derived-zoning structure you’re using.

This version is polished enough to go directly into a **portfolio**, **capstone**, or **Tableau dashboard description**.

---

# 📘 **Project Overview: Explaining Housing Cost Pressures in U.S. Cities (ACS 2021–2025)**

This project analyzes **why housing costs are rising or falling across U.S. cities** by comparing structural, economic, and demographic indicators from the **American Community Survey (ACS) 5-Year Estimates** for **2021** and **2025**.
The goal is to identify **which factors — supply, demand, affordability, or zoning constraints — most strongly explain changes in rents and home values.**

Rather than pulling hundreds of ACS variables, this project uses a **minimal, high-signal set of indicators**, organized into four analytical pillars:

1. **Housing Cost Metrics (Outcomes)**
2. **Housing Supply Indicators (Capacity, Construction & Structural Constraints)**
3. **Demand Pressure Indicators (Population, Income & Labor Market Strength)**
4. **Affordability Stress (Cost Burdens Relative to Income)**

Together, these metrics provide a comprehensive view of **how market forces and structural limitations interact to shape local housing costs.**

---

# 🏠 **1. Housing Cost Metrics (Outcomes to Explain)**

These variables represent the **dependent outcomes** the project aims to understand and model:

| Housing Cost Outcome  | ACS Table |
| --------------------- | --------- |
| **Median Gross Rent** | B25064    |
| **Median Home Value** | B25077    |

These are measured for **2021 and 2025**, allowing comparison of absolute changes and growth rates.
They form the basis for explaining **why some cities experienced rapid increases while others saw stability or decline.**

In [1]:
import pandas as pd
import sqlite3

# Load CSV files into DataFrames
MGR_2021 = pd.read_csv('Housing_Cost_Metrics/Median_Gross_Rent_2021.csv')
MGR_2024 = pd.read_csv('Housing_Cost_Metrics/Median_Gross_Rent_2024.csv')
MV_2021 = pd.read_csv('Housing_Cost_Metrics/Median_Value_2021.csv')
MV_2024 = pd.read_csv('Housing_Cost_Metrics/Median_Value_2024.csv')



# ---------------------------------------------------
# 🗃️ Load DataFrames into an SQLite in-memory database
# ---------------------------------------------------
conn = sqlite3.connect(':memory:')

MGR_2021.to_sql('Median_Gross_Rent_2021', conn, index=False, if_exists='replace')
MGR_2024.to_sql('Median_Gross_Rent_2024', conn, index=False, if_exists='replace')
MV_2021.to_sql('Median_Value_2021', conn, index=False, if_exists='replace')
MV_2024.to_sql('Median_Value_2024', conn, index=False, if_exists='replace')




# ---------------------------
# 📊 SQL Query to Join Tables
# ---------------------------
sql_query = """
SELECT DISTINCT
    g21.[Geographic Area Name],
    g21.[Median gross rent (2021)],
    g24.[Median gross rent (2024)],
    v21.[Median value (2021)],
    v24.[Median value (2024)]
FROM Median_Gross_Rent_2021 AS g21
LEFT JOIN Median_Gross_Rent_2024 AS g24
  ON g21.[Geographic Area Name] = g24.[Geographic Area Name]
LEFT JOIN Median_Value_2021 AS v21
  ON g21.[Geographic Area Name] = v21.[Geographic Area Name]
LEFT JOIN Median_Value_2024 AS v24
  ON g21.[Geographic Area Name] = v24.[Geographic Area Name];
"""
Housing_Cost_Metrics_df = pd.read_sql_query(sql_query, conn)



# Execute the query and load results into a DataFrame
Housing_Cost_Metrics_df = pd.read_sql_query(sql_query, conn)

# Save the merged DataFrame to a new CSV file
csv_filename = "Housing_Cost_Metrics_df.csv"
Housing_Cost_Metrics_df.to_csv(csv_filename, index=False)

# Close the connection
conn.close()

# Output the final DataFrame
Housing_Cost_Metrics_df

,Geographic Area Name,Median gross rent (2021),Median gross rent (2024),Median value (2021),Median value (2024)
0,"Auburn city, Alabama","1,009","1,159","319,300","392,900"
1,"Birmingham city, Alabama",895,"1,206","117,600","189,800"
2,"Dothan city, Alabama",832,"1,016","169,200","222,800"
3,"Hoover city, Alabama","1,212","1,441","363,200","424,100"
4,"Huntsville city, Alabama",983,"1,241","250,400","339,400"
...,...,...,...,...,...
629,"Caguas zona urbana, Puerto Rico",608,642,"118,200","136,400"
630,"Carolina zona urbana, Puerto Rico",645,665,"138,400","161,600"
631,"Guaynabo zona urbana, Puerto Rico",997,"1,057","213,900","257,100"
632,"Ponce zona urbana, Puerto Rico",535,495,"102,500","125,300"


# 🏗️ **2. Housing Supply Indicators (Capacity, Production & Zoning Constraints)**

Housing supply conditions are among the strongest predictors of local price changes.
This category measures **how much housing exists, how much is being built, how vacant the market is, and how flexible the city is structurally**.

### **Core Supply Indicators**

| Supply Indicator                       | ACS Table |
| -------------------------------------- | --------- |
| **Total Housing Units**                | B25001    |
| **Occupancy/Vacancy Counts**           | B25002    |
| **Units Built by Year (Construction)** | B25034    |
| **Units in Structure (Density Mix)**   | B25024    |

These metrics quantify **inventory**, **new additions to stock**, and **unused capacity**, all of which directly affect rent and home value pressures.

---

## 🧱 **Structural Zoning Proxies (Derived from Supply — No Extra Tables Needed)**

ACS does not publish zoning codes directly, but **Units in Structure (B25024)** serves as a powerful zoning constraint proxy.
From this table, the project derives:

### **Density-Based Zoning Indicators**

* **% Single-Family Detached (1-unit detached)**
* **% Small Multi-Family (2–9 units)**
* **% Large Multi-Family (10+ units)**
* **% High-Density Share (20+ units)**
* **Density Mix Index (composite structural-flexibility score)**

### **Interpretation**

Cities dominated by **single-family detached housing** tend to:

* Have restrictive zoning
* Face slower construction growth
* Exhibit higher land costs
* Experience stronger upward pressure on prices

Cities with **higher multi-family shares**:

* Absorb demand more efficiently
* See slower rent growth
* Maintain more stable housing markets

By keeping zoning proxies *inside* the Supply category, the project reflects how zoning fundamentally shapes **construction capacity, build rates, and long-term affordability**.

---


In [4]:
import pandas as pd
import sqlite3

# -----------------------------
# 1️⃣ Load CSVs
# -----------------------------
THU_2021 = pd.read_csv('Housing_Supply_Indicators/Total_Housing_Units_2021.csv')
THU_2024 = pd.read_csv('Housing_Supply_Indicators/Total_Housing_Units_2024.csv')

TOS_2021 = pd.read_csv('Housing_Supply_Indicators/Total_Occupancy_Status_2021.csv')
TOS_2024 = pd.read_csv('Housing_Supply_Indicators/Total_Occupancy_Status_2024.csv')

TV_2021 = pd.read_csv('Housing_Supply_Indicators/Total_Vacant_2021.csv')
TV_2024 = pd.read_csv('Housing_Supply_Indicators/Total_Vacant_2024.csv')

UB_2021 = pd.read_csv('Housing_Supply_Indicators/Units_Built_2021.csv')
UB_2024 = pd.read_csv('Housing_Supply_Indicators/Units_Built_2024.csv')

# Units in structure – total + buckets
UIS_TOT_2021   = pd.read_csv('Housing_Supply_Indicators/Units_in_Structure_Total_2021.csv')
UIS_TOT_2024   = pd.read_csv('Housing_Supply_Indicators/Units_in_Structure_Total_2024.csv')
UIS_DET_2021   = pd.read_csv('Housing_Supply_Indicators/Units_in_Structure_1_detached_2021.csv')
UIS_DET_2024   = pd.read_csv('Housing_Supply_Indicators/Units_in_Structure_1_detached_2024.csv')
UIS_10_19_2021 = pd.read_csv('Housing_Supply_Indicators/Units_in_Structure_10_to_19_2021.csv')
UIS_10_19_2024 = pd.read_csv('Housing_Supply_Indicators/Units_in_Structure_10_to_19_2024.csv')
UIS_20_49_2021 = pd.read_csv('Housing_Supply_Indicators/Units_in_Structure_20_to_49_2021.csv')
UIS_20_49_2024 = pd.read_csv('Housing_Supply_Indicators/Units_in_Structure_20_to_49_2024.csv')
UIS_50P_2021   = pd.read_csv('Housing_Supply_Indicators/Units_in_Structure_50_plus_2021.csv')
UIS_50P_2024   = pd.read_csv('Housing_Supply_Indicators/Units_in_Structure_50_plus_2024.csv')

# -----------------------------
# 2️⃣ Load into SQLite
# -----------------------------
conn = sqlite3.connect(":memory:")

THU_2021.to_sql("THU_2021", conn, index=False, if_exists="replace")
THU_2024.to_sql("THU_2024", conn, index=False, if_exists="replace")

TOS_2021.to_sql("TOS_2021", conn, index=False, if_exists="replace")
TOS_2024.to_sql("TOS_2024", conn, index=False, if_exists="replace")

TV_2021.to_sql("TV_2021", conn, index=False, if_exists="replace")
TV_2024.to_sql("TV_2024", conn, index=False, if_exists="replace")

UB_2021.to_sql("UB_2021", conn, index=False, if_exists="replace")
UB_2024.to_sql("UB_2024", conn, index=False, if_exists="replace")

UIS_TOT_2021.to_sql("UIS_TOT_2021", conn, index=False, if_exists="replace")
UIS_TOT_2024.to_sql("UIS_TOT_2024", conn, index=False, if_exists="replace")
UIS_DET_2021.to_sql("UIS_DET_2021", conn, index=False, if_exists="replace")
UIS_DET_2024.to_sql("UIS_DET_2024", conn, index=False, if_exists="replace")
UIS_10_19_2021.to_sql("UIS_10_19_2021", conn, index=False, if_exists="replace")
UIS_10_19_2024.to_sql("UIS_10_19_2024", conn, index=False, if_exists="replace")
UIS_20_49_2021.to_sql("UIS_20_49_2021", conn, index=False, if_exists="replace")
UIS_20_49_2024.to_sql("UIS_20_49_2024", conn, index=False, if_exists="replace")
UIS_50P_2021.to_sql("UIS_50P_2021", conn, index=False, if_exists="replace")
UIS_50P_2024.to_sql("UIS_50P_2024", conn, index=False, if_exists="replace")

# -----------------------------
# 3️⃣ SQL Join (uses your column names)
# -----------------------------
sql_query = """
SELECT DISTINCT
    thu21.[Geographic Area Name] AS City,

    -- Totals
    thu21.[Total Units (2021)]                 AS Total_Units_2021,
    thu24.[Total Units (2024)]                 AS Total_Units_2024,

    tos21.[Total Occupancy Status (2021)]      AS Occupancy_2021,
    tos24.[Total Occupancy Status (2024)]      AS Occupancy_2024,

    tv21.[Vacant (2021)]                       AS Vacant_2021,
    tv24.[Vacant (2024)]                       AS Vacant_2024,

    ub21.[Units Built (2021)]                  AS Units_Built_2021,
    ub24.[Units Built (2024)]                  AS Units_Built_2024,

    -- Units in structure – total
    uis_tot21.[Units in Structure (2021)]      AS UIS_Total_2021,
    uis_tot24.[Units in Structure (2024)]      AS UIS_Total_2024,

    -- Buckets
    uis_det21.[1, detached (2021)]             AS Detached_1_Unit_2021,
    uis_det24.[1, detached (2024)]             AS Detached_1_Unit_2024,

    uis_10_19_21.[10 to 19 (2021)]             AS Units_10_19_2021,
    uis_10_19_24.[10 to 19 (2024)]             AS Units_10_19_2024,

    uis_20_49_21.[20 to 49 (2021)]             AS Units_20_49_2021,
    uis_20_49_24.[20 to 49 (2024)]             AS Units_20_49_2024,

    uis_50p_21.[50 or more (2021)]             AS Units_50plus_2021,
    uis_50p_24.[50 or more (2024)]             AS Units_50plus_2024

FROM THU_2021 AS thu21
LEFT JOIN THU_2024       AS thu24       ON thu21.[Geographic Area Name] = thu24.[Geographic Area Name]
LEFT JOIN TOS_2021       AS tos21       ON thu21.[Geographic Area Name] = tos21.[Geographic Area Name]
LEFT JOIN TOS_2024       AS tos24       ON thu21.[Geographic Area Name] = tos24.[Geographic Area Name]
LEFT JOIN TV_2021        AS tv21        ON thu21.[Geographic Area Name] = tv21.[Geographic Area Name]
LEFT JOIN TV_2024        AS tv24        ON thu21.[Geographic Area Name] = tv24.[Geographic Area Name]
LEFT JOIN UB_2021        AS ub21        ON thu21.[Geographic Area Name] = ub21.[Geographic Area Name]
LEFT JOIN UB_2024        AS ub24        ON thu21.[Geographic Area Name] = ub24.[Geographic Area Name]
LEFT JOIN UIS_TOT_2021   AS uis_tot21   ON thu21.[Geographic Area Name] = uis_tot21.[Geographic Area Name]
LEFT JOIN UIS_TOT_2024   AS uis_tot24   ON thu21.[Geographic Area Name] = uis_tot24.[Geographic Area Name]
LEFT JOIN UIS_DET_2021   AS uis_det21   ON thu21.[Geographic Area Name] = uis_det21.[Geographic Area Name]
LEFT JOIN UIS_DET_2024   AS uis_det24   ON thu21.[Geographic Area Name] = uis_det24.[Geographic Area Name]
LEFT JOIN UIS_10_19_2021 AS uis_10_19_21 ON thu21.[Geographic Area Name] = uis_10_19_21.[Geographic Area Name]
LEFT JOIN UIS_10_19_2024 AS uis_10_19_24 ON thu21.[Geographic Area Name] = uis_10_19_24.[Geographic Area Name]
LEFT JOIN UIS_20_49_2021 AS uis_20_49_21 ON thu21.[Geographic Area Name] = uis_20_49_21.[Geographic Area Name]
LEFT JOIN UIS_20_49_2024 AS uis_20_49_24 ON thu21.[Geographic Area Name] = uis_20_49_24.[Geographic Area Name]
LEFT JOIN UIS_50P_2021   AS uis_50p_21   ON thu21.[Geographic Area Name] = uis_50p_21.[Geographic Area Name]
LEFT JOIN UIS_50P_2024   AS uis_50p_24   ON thu21.[Geographic Area Name] = uis_50p_24.[Geographic Area Name];
"""

Housing_Supply_Indicators_df = pd.read_sql_query(sql_query, conn)

# -----------------------------
# 4️⃣ Save / inspect
# -----------------------------
Housing_Supply_Indicators_df.to_csv("Housing_Supply_Indicators_df.csv", index=False)
conn.close()

Housing_Supply_Indicators_df


,City,Total_Units_2021,Total_Units_2024,Occupancy_2021,Occupancy_2024,Vacant_2021,Vacant_2024,Units_Built_2021,Units_Built_2024,UIS_Total_2021,UIS_Total_2024,Detached_1_Unit_2021,Detached_1_Unit_2024,Units_10_19_2021,Units_10_19_2024,Units_20_49_2021,Units_20_49_2024,Units_50plus_2021,Units_50plus_2024
0,"Auburn city, Alabama","35,161","36,889","35,161","36,889","6,025","6,883","35,161","36,889","35,161","36,889","16,474","17,351","5,100","4,138","1,503","2,656","1,289","3,340"
1,"Birmingham city, Alabama","108,533","110,757","108,533","110,757","20,963","19,627","108,533","110,757","108,533","110,757","64,470","66,117","8,641","8,540","6,875","6,977","6,644","10,513"
2,"Dothan city, Alabama","33,868","34,695","33,868","34,695","3,567","3,967","33,868","34,695","33,868","34,695","24,364","24,717","1,353","1,234",626,293,402,815
3,"Hoover city, Alabama","38,821","41,940","38,821","41,940","2,100","2,088","38,821","41,940","38,821","41,940","26,413","28,453","4,118","4,426","1,632","1,315",922,"1,890"
4,"Huntsville city, Alabama","104,141","106,466","104,141","106,466","7,590","9,664","104,141","106,466","104,141","106,466","69,906","69,001","6,906","6,851","3,288","2,951","5,469","5,428"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629,"Caguas zona urbana, Puerto Rico","35,851","38,750","35,851","38,750","5,881","11,638","35,851","38,750","35,851","38,750","20,134","21,693","1,808","1,833",362,"1,325",907,"1,915"
630,"Carolina zona urbana, Puerto Rico","68,134","72,721","68,134","72,721","14,122","15,864","68,134","72,721","68,134","72,721","31,086","32,587","5,042","4,477","2,264","2,512","8,559","10,274"
631,"Guaynabo zona urbana, Puerto Rico","31,709","31,935","31,709","31,935","4,730","4,530","31,709","31,935","31,709","31,935","16,915","16,327","2,064",612,"1,403","1,572","4,255","6,101"
632,"Ponce zona urbana, Puerto Rico","55,535","57,739","55,535","57,739","13,131","11,105","55,535","57,739","55,535","57,739","39,333","39,282","3,520","4,064","1,107","1,831","1,379","1,458"


# 📈 **3. Demand Pressure Indicators (Population, Income & Labor Market Strength)**

Demand-side pressures raise prices when **population or purchasing power grows faster than housing supply**.

| Demand Indicator                        | ACS Table |
| --------------------------------------- | --------- |
| **Total Population**                    | B01003    |
| **Median Household Income**             | B19013    |
| **Labor Force & Employment Indicators** | S2301     |

### Why Demand Matters

* Growing populations require more units than cities may be providing.
* Rising incomes can **bid up** rents and home values.
* High employment and strong labor markets bring new workers — and greater housing competition.

This category helps measure **how much pressure is being placed on existing housing stock.**

---

In [5]:
import pandas as pd
import sqlite3

# -----------------------------
# 1️⃣ Load CSVs
# -----------------------------
EPR_2021 = pd.read_csv("Pressure_Indicators/Employment_Population_Ratio_2021.csv")
# note: filename has a typo in screenshot: "Ration" – using that so it matches disk
EPR_2024 = pd.read_csv("Pressure_Indicators/Employment_Population_Ration_2024.csv")

LFP_2021 = pd.read_csv("Pressure_Indicators/Labor_Force_Participation_Rate_2021.csv")
LFP_2024 = pd.read_csv("Pressure_Indicators/Labor_Force_Participation_Rate_2024.csv")

MHI_2021 = pd.read_csv("Pressure_Indicators/Median_Household_Income_2021.csv")
MHI_2024 = pd.read_csv("Pressure_Indicators/Median_Household_Income_2024.csv")

POP_2021 = pd.read_csv("Pressure_Indicators/Population_2021.csv")
POP_2024 = pd.read_csv("Pressure_Indicators/Population_2024.csv")

UR_2021 = pd.read_csv("Pressure_Indicators/Unemployment_Rate_2021.csv")
UR_2024 = pd.read_csv("Pressure_Indicators/Unemployment_Rate_2024.csv")

# -----------------------------
# 2️⃣ Load into SQLite
# -----------------------------
conn = sqlite3.connect(":memory:")

EPR_2021.to_sql("EPR_2021", conn, index=False, if_exists="replace")
EPR_2024.to_sql("EPR_2024", conn, index=False, if_exists="replace")

LFP_2021.to_sql("LFP_2021", conn, index=False, if_exists="replace")
LFP_2024.to_sql("LFP_2024", conn, index=False, if_exists="replace")

MHI_2021.to_sql("MHI_2021", conn, index=False, if_exists="replace")
MHI_2024.to_sql("MHI_2024", conn, index=False, if_exists="replace")

POP_2021.to_sql("POP_2021", conn, index=False, if_exists="replace")
POP_2024.to_sql("POP_2024", conn, index=False, if_exists="replace")

UR_2021.to_sql("UR_2021", conn, index=False, if_exists="replace")
UR_2024.to_sql("UR_2024", conn, index=False, if_exists="replace")

# -----------------------------
# 3️⃣ SQL Join (Pressure Indicators)
# -----------------------------
sql_query = """
SELECT DISTINCT
    pop21.[Geographic Area Name]                         AS City,

    -- Population
    pop21.[Population (2021)]                            AS Population_2021,
    pop24.[Population (2024)]                            AS Population_2024,

    -- Employment / Population Ratio
    epr21.[Employment/Population Ratio (2021)]          AS EmpPopRatio_2021,
    epr24.[Employment/Population Ratio (2024)]          AS EmpPopRatio_2024,

    -- Labor Force Participation Rate
    lfp21.[Labor Force Participation Rate (2021)]       AS LFPR_2021,
    lfp24.[Labor Force Participation Rate (2024)]       AS LFPR_2024,

    -- Median Household Income
    mhi21.[Median household income (2021)]              AS Median_HH_Income_2021,
    mhi24.[Median household income (2024)]              AS Median_HH_Income_2024,

    -- Unemployment Rate
    ur21.[Unemployment Rate (2021)]                     AS Unemployment_Rate_2021,
    ur24.[Unemployment Rate (2024)]                     AS Unemployment_Rate_2024

FROM POP_2021 AS pop21
LEFT JOIN POP_2024 AS pop24
       ON pop21.[Geographic Area Name] = pop24.[Geographic Area Name]
LEFT JOIN EPR_2021 AS epr21
       ON pop21.[Geographic Area Name] = epr21.[Geographic Area Name]
LEFT JOIN EPR_2024 AS epr24
       ON pop21.[Geographic Area Name] = epr24.[Geographic Area Name]
LEFT JOIN LFP_2021 AS lfp21
       ON pop21.[Geographic Area Name] = lfp21.[Geographic Area Name]
LEFT JOIN LFP_2024 AS lfp24
       ON pop21.[Geographic Area Name] = lfp24.[Geographic Area Name]
LEFT JOIN MHI_2021 AS mhi21
       ON pop21.[Geographic Area Name] = mhi21.[Geographic Area Name]
LEFT JOIN MHI_2024 AS mhi24
       ON pop21.[Geographic Area Name] = mhi24.[Geographic Area Name]
LEFT JOIN UR_2021 AS ur21
       ON pop21.[Geographic Area Name] = ur21.[Geographic Area Name]
LEFT JOIN UR_2024 AS ur24
       ON pop21.[Geographic Area Name] = ur24.[Geographic Area Name];
"""

Pressure_Indicators_df = pd.read_sql_query(sql_query, conn)

# -----------------------------
# 4️⃣ Save / inspect
# -----------------------------
Pressure_Indicators_df.to_csv("Pressure_Indicators_df.csv", index=False)
conn.close()

Pressure_Indicators_df


,City,Population_2021,Population_2024,EmpPopRatio_2021,EmpPopRatio_2024,LFPR_2021,LFPR_2024,Median_HH_Income_2021,Median_HH_Income_2024,Unemployment_Rate_2021,Unemployment_Rate_2024
0,"Auburn city, Alabama","78,552","83,761",None,None,None,None,"48,531","69,615",None,None
1,"Birmingham city, Alabama","196,410","198,477",54.2%,58.3%,58.9%,61.3%,"36,614","51,355",7.4%,4.6%
2,"Dothan city, Alabama","71,283","72,735",53.3%,53.4%,56.6%,57.3%,"45,088","55,846",5.3%,5.5%
3,"Hoover city, Alabama","92,588","93,002",66.4%,None,68.9%,None,"99,276","106,844",3.5%,None
4,"Huntsville city, Alabama","215,482","230,157",59.5%,64.0%,62.2%,66.5%,"70,757","83,235",3.9%,3.3%
...,...,...,...,...,...,...,...,...,...,...,...
629,"Caguas zona urbana, Puerto Rico","76,577","74,695",42.5%,None,46.4%,None,"22,125","28,547",8.6%,None
630,"Carolina zona urbana, Puerto Rico","141,050","136,303",45.6%,49.8%,54.0%,53.9%,"31,597","36,963",14.2%,6.8%
631,"Guaynabo zona urbana, Puerto Rico","72,088","68,908",46.3%,55.5%,50.7%,56.6%,"53,479","58,141",7.8%,2.1%
632,"Ponce zona urbana, Puerto Rico","112,005","101,024",34.7%,37.4%,41.0%,43.8%,"17,207","22,061",15.2%,14.4%


# 💸 **4. Affordability Stress (Housing Costs Relative to Income)**

Affordability indicators capture how comfortably residents can cover their housing expenses.

| Affordability Indicator                | ACS Table |
| -------------------------------------- | --------- |
| **Gross Rent as % of Income**          | B25070    |
| **Total Cost-Burdened Renters**        | B25070    |
| **Severely Burdened (≥50% of income)** | Derived   |

### Why This Matters

Even if rents rise modestly, affordability may deteriorate if:

* Incomes stagnate
* Low-income renters face disproportionate increases
* Housing supply is severely constrained

This category quantifies **household stress and vulnerability**, a critical dimension for policymakers and analysts.

---


In [6]:
import pandas as pd
import sqlite3

# -----------------------------
# 1️⃣ Load CSVs
# -----------------------------
PCT_30_35_2021 = pd.read_csv("Affordability_Stress/30_to_35_Pct_2021.csv")
PCT_30_35_2024 = pd.read_csv("Affordability_Stress/30_to_35_Pct_2024.csv")

PCT_35_40_2021 = pd.read_csv("Affordability_Stress/35_to_40_Pct_2021.csv")
PCT_35_40_2024 = pd.read_csv("Affordability_Stress/35_to_40_Pct_2024.csv")

PCT_40_50_2021 = pd.read_csv("Affordability_Stress/40_to_50_Pct_2021.csv")
PCT_40_50_2024 = pd.read_csv("Affordability_Stress/40_to_50_Pct_2024.csv")

PCT_50P_2021   = pd.read_csv("Affordability_Stress/50_or_More_Pct_2021.csv")
PCT_50P_2024   = pd.read_csv("Affordability_Stress/50_or_More_Pct_2024.csv")

TOT_BURD_2021  = pd.read_csv("Affordability_Stress/Total_Cost_Burden_2021.csv")
TOT_BURD_2024  = pd.read_csv("Affordability_Stress/Total_Cost_Burden_2024.csv")

# -----------------------------
# 2️⃣ Load into SQLite
# -----------------------------
conn = sqlite3.connect(":memory:")

PCT_30_35_2021.to_sql("PCT_30_35_2021", conn, index=False, if_exists="replace")
PCT_30_35_2024.to_sql("PCT_30_35_2024", conn, index=False, if_exists="replace")

PCT_35_40_2021.to_sql("PCT_35_40_2021", conn, index=False, if_exists="replace")
PCT_35_40_2024.to_sql("PCT_35_40_2024", conn, index=False, if_exists="replace")

PCT_40_50_2021.to_sql("PCT_40_50_2021", conn, index=False, if_exists="replace")
PCT_40_50_2024.to_sql("PCT_40_50_2024", conn, index=False, if_exists="replace")

PCT_50P_2021.to_sql("PCT_50P_2021", conn, index=False, if_exists="replace")
PCT_50P_2024.to_sql("PCT_50P_2024", conn, index=False, if_exists="replace")

TOT_BURD_2021.to_sql("TOT_BURD_2021", conn, index=False, if_exists="replace")
TOT_BURD_2024.to_sql("TOT_BURD_2024", conn, index=False, if_exists="replace")

# -----------------------------
# 3️⃣ SQL Join – Affordability Stress
# -----------------------------
sql_query = """
SELECT DISTINCT
    t21.[Geographic Area Name] AS City,

    -- Buckets of rent burden
    p30_21.[30.0 to 34.9 percent (2021)] AS Pct_30_35_2021,
    p30_24.[30.0 to 34.9 percent (2024)] AS Pct_30_35_2024,

    p35_21.[35.0 to 39.9 percent (2021)] AS Pct_35_40_2021,
    p35_24.[35.0 to 39.9 percent (2024)] AS Pct_35_40_2024,

    p40_21.[40.0 to 49.9 percent (2021)] AS Pct_40_50_2021,
    p40_24.[40.0 to 49.9 percent (2024)] AS Pct_40_50_2024,

    p50_21.[50.0 percent or more (2021)] AS Pct_50plus_2021,
    p50_24.[50.0 percent or more (2024)] AS Pct_50plus_2024,

    -- Total cost-burdened renters (>= 30%)
    t21.[Total Burdened (2021)]         AS Total_Burdened_2021,
    t24.[Total Burdened (2024)]         AS Total_Burdened_2024

FROM TOT_BURD_2021 AS t21
LEFT JOIN TOT_BURD_2024   AS t24   ON t21.[Geographic Area Name] = t24.[Geographic Area Name]
LEFT JOIN PCT_30_35_2021  AS p30_21 ON t21.[Geographic Area Name] = p30_21.[Geographic Area Name]
LEFT JOIN PCT_30_35_2024  AS p30_24 ON t21.[Geographic Area Name] = p30_24.[Geographic Area Name]
LEFT JOIN PCT_35_40_2021  AS p35_21 ON t21.[Geographic Area Name] = p35_21.[Geographic Area Name]
LEFT JOIN PCT_35_40_2024  AS p35_24 ON t21.[Geographic Area Name] = p35_24.[Geographic Area Name]
LEFT JOIN PCT_40_50_2021  AS p40_21 ON t21.[Geographic Area Name] = p40_21.[Geographic Area Name]
LEFT JOIN PCT_40_50_2024  AS p40_24 ON t21.[Geographic Area Name] = p40_24.[Geographic Area Name]
LEFT JOIN PCT_50P_2021    AS p50_21 ON t21.[Geographic Area Name] = p50_21.[Geographic Area Name]
LEFT JOIN PCT_50P_2024    AS p50_24 ON t21.[Geographic Area Name] = p50_24.[Geographic Area Name];
"""

Affordability_Stress_df = pd.read_sql_query(sql_query, conn)

# -----------------------------
# 4️⃣ Save / inspect
# -----------------------------
Affordability_Stress_df.to_csv("Affordability_Stress_df.csv", index=False)
conn.close()

Affordability_Stress_df


,City,Pct_30_35_2021,Pct_30_35_2024,Pct_35_40_2021,Pct_35_40_2024,Pct_40_50_2021,Pct_40_50_2024,Pct_50plus_2021,Pct_50plus_2024,Total_Burdened_2021,Total_Burdened_2024
0,"Auburn city, Alabama","1,006",949,444,679,"1,357",654,"6,495","3,709","14,300","15,118"
1,"Birmingham city, Alabama","3,751","5,633","4,553","2,543","4,219","3,441","12,366","11,688","48,444","46,905"
2,"Dothan city, Alabama","1,392","1,153",677,"1,099","1,091","1,048","3,134","2,639","12,638","13,583"
3,"Hoover city, Alabama",741,888,274,229,846,"2,657","2,875","1,231","11,040","10,728"
4,"Huntsville city, Alabama","2,628","2,158","3,112","1,873","3,245","2,776","7,194","8,115","41,110","37,536"
...,...,...,...,...,...,...,...,...,...,...,...
641,"Caguas zona urbana, Puerto Rico",822,903,113,264,"1,253",226,"2,680","1,186","11,010","12,005"
642,"Carolina zona urbana, Puerto Rico",327,"1,908","1,010",906,746,793,"3,930","2,421","19,929","18,628"
643,"Guaynabo zona urbana, Puerto Rico",747,141,0,258,64,235,"1,546","1,208","5,599","6,698"
644,"Ponce zona urbana, Puerto Rico","1,268","1,450",446,673,476,"1,226","2,772","1,778","16,000","13,276"


# ✔️ **Project Summary**

This streamlined dataset enables a clear, multi-dimensional analysis of why housing costs differ across cities — and why they change over time.

The combined indicators allow us to diagnose whether local price shifts are driven by:

### 🔹 **Insufficient supply or restrictive zoning**

Low vacancy, slow building, and single-family dominance signal constrained markets.

### 🔹 **Surging demand from population or income growth**

Demand expands faster than housing — prices rise.

### 🔹 **Labor market strength or economic resilience**

Higher employment → more households → more competition.

### 🔹 **Affordability deterioration**

Housing costs rise faster than incomes → more burdened residents.

### 🔹 **Structural flexibility of the housing stock**

Cities with high-density multi-family units absorb shocks better.

By comparing **2021 vs. 2024**, the project identifies **which cities face the most pressure**, which are stabilizing, and which structural factors explain trends in housing cost outcomes.

---
